In [1]:
import findspark
findspark.init('/home/nono/spark-2.4.0-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [14]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App', 'Time on Website', 'Length of Membership', 'Yearly Amount Spent'], outputCol='features')

In [15]:
output = assembler.transform(data)

In [16]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [17]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [19]:
final_data = output.select('features', 'Yearly Amount Spent')

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [21]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [22]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                339|
|   mean| 497.49017023650606|
| stddev|  80.30997926596666|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [23]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [24]:
lr_model = lr.fit(train_data)

In [25]:
test_results = lr_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-1.08002495835535...|
|5.684341886080801...|
|-5.68434188608080...|
|3.979039320256561...|
|2.046363078989088...|
|5.684341886080801...|
|-5.68434188608080...|
|1.136868377216160...|
|-1.76214598468504...|
|5.115907697472721...|
|-2.84217094304040...|
|3.979039320256561...|
|1.080024958355352...|
|6.252776074688882...|
|1.648459146963432...|
|3.410605131648481...|
|-4.54747350886464...|
|-1.64845914696343...|
|-5.68434188608080...|
|-1.13686837721616...|
+--------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

9.652679496568683e-13

In [28]:
test_results.r2

1.0

In [31]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [32]:
unlabeled_data = test_data.select('features')

In [33]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.4925366965402...|
|[30.5743636841713...|
|[30.8364326747734...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.3895854806643...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.7366356860502...|
|[31.8124825597242...|
|[31.8648325480987...|
|[31.8745516945853...|
|[31.9048571310136...|
|[31.9453957483445...|
|[31.9764800614612...|
|[32.0180740106320...|
|[32.0215955013870...|
|[32.0444861274404...|
+--------------------+
only showing top 20 rows



In [34]:
prediction = lr_model.transform(unlabeled_data)

In [35]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|408.64035107262856|
|[30.4925366965402...|282.47124571991395|
|[30.5743636841713...| 442.0644137580657|
|[30.8364326747734...| 467.5019004269892|
|[31.1239743499119...| 486.9470538397637|
|[31.1280900496166...| 557.2526867470541|
|[31.3895854806643...|410.06961105998295|
|[31.5761319713222...| 541.2265839893282|
|[31.6005122003032...| 479.1728514910987|
|[31.6253601348306...| 376.3369007569237|
|[31.7366356860502...| 496.9334462555322|
|[31.8124825597242...| 392.8103449837968|
|[31.8648325480987...| 439.8912804768126|
|[31.8745516945853...|392.28524424626687|
|[31.9048571310136...| 473.9498574228145|
|[31.9453957483445...| 657.0199239376516|
|[31.9764800614612...|330.59444603410066|
|[32.0180740106320...|357.78311074531695|
|[32.0215955013870...|  521.572174757828|
|[32.0444861274404...| 448.2298291865501|
+--------------------+------------